In [1]:
from owlready2 import *

onto = get_ontology("https://spec.industrialontologies.org/ontology/hbom/")
core_onto = get_ontology("https://raw.githubusercontent.com/iofoundry/ontology/refs/heads/slot/core/Core.rdf").load()
pps_onto = get_ontology("ProductionPlanning.rdf").load()

onto.imported_ontologies.append(core_onto)
onto.imported_ontologies.append(pps_onto)

print(list(onto.imported_ontologies.first().classes()))
print(list(onto.imported_ontologies[1].classes()))

ns = onto.get_namespace("https://spec.industrialontologies.org/ontology/testaction/")
core = onto.get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
bfo = onto.get_namespace("http://purl.obolibrary.org/obo/")
pps = onto.get_namespace("https://spec.industrialontologies.org/ontology/productionplanning/ProductionPlanning/")



[Core.InformationContentEntity, Core.EngineeredSystem, Core.GroupOfAgents, Core.Person, Core.Capability, Core.DescriptiveInformationContentEntity, Core.DesignativeInformationContentEntity, Core.DirectiveInformationContentEntity, Core.InformationSlot, Core.ValueExpression, Core.ProcessCharacteristic, Core.PlannedProcess, Core.Agent, Core.RequirementSpecification, Core.TemporalInstantValueExpression, Core.ActionSpecification, Core.Adhesive, Core.JoiningMaterial, Core.AgentRole, Core.AggregationDescription, Core.CountSlot, Core.Agreement, Core.ObjectiveSpecification, Core.Algorithm, Core.Assembly, Core.MaterialArtifact, Core.MaterialComponent, Core.AssemblyProcess, Core.ManufacturingProcess, Core.AssemblySpecification, Core.DesignSpecification, Core.BusinessFunction, Core.Organization, Core.BusinessProcess, Core.BusinessOrganization, Core.PlanSpecification, Core.Buyer, Core.BuyerRole, Core.BuyingBusinessProcess, Core.CommercialServiceAgreement, Core.MaterialProduct, Core.CommercialService

In [2]:
bfo.BFO_0000063.python_name = "precedes"
bfo.BFO_0000118.python_name = "hasProperOccurrentPart"
pps.hasResource.python_name = "hasResource"
core.describedBy.python_name = "describedBy"
pps.hasActionSpecification.python_name = "hasActionSpecification" 
pps.hasObjectiveSpecification.python_name = "hasObjectiveSpecification"

with onto:

    # operation specifiactions
    i_operation1Spec = pps.OperationSpecification("operation1-spec")
    i_operation2Spec = pps.OperationSpecification("operation2-spec")
    i_operation3Spec = pps.OperationSpecification("operation3-spec")    

    i_processPlan = pps.ProcessPlan("process-plan1")
    i_processPlan.hasActionSpecification.append(i_operation1Spec)
    i_processPlan.hasActionSpecification.append(i_operation2Spec)
    i_processPlan.hasActionSpecification.append(i_operation3Spec)



    # create machines
    i_machine1 = pps.ProductionMachine("cnc-mill")
    i_machine2 = pps.ProductionMachine("hydralic-press")
    i_machine3 = pps.ProductionMachine("laser-marker")
    # create tools
    i_drillbit = pps.ProductionTool("twist-drill-bit")
    i_endmill = pps.ProductionTool("carbide-end-mill")

    # machine 1 processes
    i_setup1Machine1 = pps.SetupProcess("setup1-on-machine1")

    drilling = types.new_class("DrillingProcess", (core.ManufacturingProcess,)) #create a drilling class
    i_drilling = drilling("drilling")
    i_drilling.hasResource.append(i_machine1)
    i_drilling.hasResource.append(i_drillbit)
    
    i_setup2Machine1 = pps.SetupProcess("setup2-on-machine1")
    
    milling = types.new_class("MillingProcess", (core.ManufacturingProcess,)) #create a milling class
    i_milling = milling("milling")
    i_milling.hasResource.append(i_machine1)
    i_milling.hasResource.append(i_endmill)

    i_inspection1Machine1 = pps.InspectionProcess("inspection1-after-cutting")
    
    i_transfer1 = pps.MachineTransferProcess("transfer1-to-press")

    # step 1 on machine 1
    i_step1machine1 = pps.ProcessingStep("step1-on-machine1")
    i_step1machine1.hasProperOccurrentPart.append(i_setup1Machine1)
    i_step1machine1.hasProperOccurrentPart.append(i_drilling)

    # step 2 on machine 1
    i_step2machine1 = pps.ProcessingStep("step2-on-machine1")
    i_step2machine1.hasProperOccurrentPart.append(i_setup2Machine1)
    i_step2machine1.hasProperOccurrentPart.append(i_milling)
    i_step2machine1.hasProperOccurrentPart.append(i_inspection1Machine1)

    # operation 1
    i_operation1 = pps.ManufacturingOperation("operation1")
    i_operation1.hasProperOccurrentPart.append(i_step1machine1)
    i_operation1.hasProperOccurrentPart.append(i_step2machine1)
    i_operation1.describedBy.append(i_operation1Spec)
    
    # machine 2 processes
    i_setup1Machine2 = pps.SetupProcess("setup1-on-machine2")

    pressBraking = types.new_class("PressingProcess", (core.ManufacturingProcess,)) #create a press braking class
    i_pressBraking = pressBraking("press-bending")
    i_pressBraking.hasResource.append(i_machine2)

    i_inspection1Machine2 = pps.InspectionProcess("inspection1-afte-pressing")

    i_transfer2 = pps.MachineTransferProcess("transfer-to-marking")

    # step 1 on machine 2
    i_step1machine2 = pps.ProcessingStep("step1-on-machine2")
    i_step1machine2.hasProperOccurrentPart.append(i_setup1Machine2)
    i_step1machine2.hasProperOccurrentPart.append(i_pressBraking)
    i_step1machine2.hasProperOccurrentPart.append(i_inspection1Machine2)

    # operation 2
    i_operation2 = pps.ManufacturingOperation("operation2")
    i_operation2.hasProperOccurrentPart.append(i_step1machine2)
    i_operation2.describedBy.append(i_operation2Spec)

    # machine 3 processes
    laserMarking = types.new_class("LaserMarkingProcess", (core.ManufacturingProcess,)) #create a press braking class
    i_laserMarking = pressBraking("laser-marking")
    i_laserMarking.hasResource.append(i_machine3)    

    i_inspection1Machine3 = pps.InspectionProcess("final-inspection")

    # step 1 on machine 3
    i_step1machine3 = pps.ProcessingStep("step1-on-machine3")
    i_step1machine3.hasProperOccurrentPart.append(i_laserMarking)

    # operation 3
    i_operation3 = pps.ManufacturingOperation("operation3")
    i_operation3.hasProperOccurrentPart.append(i_step1machine3)
    i_operation3.describedBy.append(i_operation3Spec)

    # sequence the processes
    i_setup1Machine1.precedes.append(i_drilling)
    i_drilling.precedes.append(i_setup2Machine1)
    i_setup2Machine1.precedes.append(i_milling)
    i_milling.precedes.append(i_inspection1Machine1)
    i_inspection1Machine1.precedes.append(i_transfer1)
    i_transfer1.precedes.append(i_setup1Machine2)
    i_setup1Machine2.precedes.append(i_pressBraking)
    i_pressBraking.precedes.append(i_inspection1Machine2)
    i_inspection1Machine2.precedes.append(i_transfer2)
    i_transfer2.precedes.append(i_laserMarking)
    i_laserMarking.precedes.append(i_inspection1Machine3)

onto.save(file = "production-plan.rdf", format = "rdfxml")